# Run time

Tests complete in slightly less than 1 minute on a single core.

# Purpose

We now have bindings to the special variance-constrained semi-grand canonical (VCSGC) Monte Carlo/MD coupling implemented [here](https://vcsgc-lammps.materialsmodeling.org). This is a simple set of tests to make sure that it is working smoothly.

In [ ]:
from pyiron.project import Project 
import numpy as np

In [ ]:
pr = Project('integration_test_Lammps_vcsgc')
pr.remove_jobs(recursive=True)

In [ ]:
# Create job
job = pr.create_job(pr.job_type.Lammps, 'vcsgc_job')
job.structure = pr.create_ase_bulk('Al', cubic=True).repeat(15)  # VCSGC has minimial cell size requirements
job.structure[0] = 'Mg'  # Temporarily set one atom to Mg so we can programatically get a binary potential
job.potential = job.list_potentials()[0]
job.structure[0] = 'Al'  # And switch it back so we start with pure Al

# Define chemical potentials
mu = {s: 0. for s in job.input.potential.get_element_lst()}
mu['Mg'] = -100  # Massive preference for Mg

# Set target concentrations for 50-50 Al-Mg regardless of whatever else is in the potential
target_concentration = dict(mu)
target_concentration['Al'] = 0.5
target_concentration['Mg'] = 0.5

# Run VCSGC
job.calc_vcsgc(
    mu=mu,
    target_concentration=target_concentration,
    kappa=1000000.,  # Very large because we want to hit 50-50 exactly and quickly and mu_Mg is strong
    mc_step_interval=1,
    swap_fraction=1.,
    n_ionic_steps=2,
    n_print=2,
    temperature=300,
    pressure=None
)
job.run()

# Check that the system ends in a 50-50 state
symbols = np.array([el.Abbreviation for el in job.get_structure(iteration_step=-1).elements])
assert np.sum(symbols == 'Mg') == np.sum(symbols == 'Al'), "The system was not equiatomic"

In [ ]:
# Clean up
pr.remove_jobs(recursive=True)
pr.remove(enable=True)